In [ ]:
# Parameters
TENANT_ID = None
REGION = None
INVESTIGATION_TITLE = None
TAEGIS_MAGIC_NOTEBOOK_FILENAME = None

# Report Title

Write your description of what you are looking for and why it would be an issue.

## Executive Summary

1-2 sentences giving an overview of what happened to which resources.

In [ ]:
%load_ext taegis_magic

from taegis_magic.pandas.alerts import (
    inflate_raw_events,
    convert_alert_timestamps,
    severity_rounded_and_category,
    normalize_creator_name,
)
from taegis_magic.pandas.events import convert_event_timestamps

## Evidence and Analysis

### Alerts

State what the alerts search is looking for and what to do with the findings.

In [ ]:
# https://docs.ctpx.secureworks.com/search/querylanguage/advanced_search/

In [ ]:
%%taegis alerts search --tenant $TENANT_ID --region $REGION --assign alerts --track
FROM alert
EARLIEST=-1d | head 10

In [ ]:
# check for an empty DataFrame, otherwise the notebook may fail with null results
if alerts.empty:
    transformed_alerts = alerts
else:
    transformed_alerts = (
        alerts.pipe(inflate_raw_events)
        .pipe(convert_alert_timestamps)
        .pipe(severity_rounded_and_category)
        .pipe(normalize_creator_name, region=REGION)
    )

In [ ]:
# transformed_alerts.columns

In [ ]:
# columns may differ from your data, please check the columns above to validate
if not transformed_alerts.empty:
    transformed_alerts[
        [
            "taegis_magic.metadata.inserted_at.seconds",
            "taegis_magic.creator.display_name",
            "metadata.title",
            "status",
            "taegis_magic.severity_category",
            "event_data.resource_id",
        ]
    ].value_counts()

In [ ]:
# alerts_filtered = transformed_alerts[transformed_alerts["metadata.severity"] >= 0.8]

In [ ]:
# %taegis investigations evidence stage alerts alerts_filtered

### Events

State what the events search is looking for and what do do with the findings.

In [ ]:
%%taegis events search --tenant $TENANT_ID --region $REGION --assign processes --track
FROM process
EARLIEST=-1d | head 10

In [ ]:
# check for an empty DataFrame, otherwise the notebook may fail with null results
if processes.empty:
    transformed_processes = processes
else:
    transformed_processes = processes.pipe(convert_event_timestamps)

In [ ]:
# transformed_processes.columns

In [ ]:
# columns may differ from your data, please check the columns above to validate
if not transformed_processes.empty:
    transformed_processes[
        [
            "hostname",
            "host_id",
            "event_time_usec",
            "process_id",
            "parent_process_id",
            "commandline",
        ]
    ]

In [ ]:
# filter processes here
if not transformed_processes.empty:
    processes_filtered = transformed_processes[
        transformed_processes["commandline"].str.contains("cmd.exe")
    ]

In [ ]:
# %taegis investigations evidence stage events processes_filtered

# Report Findings

Put your findings here.

In [ ]:
%save_notebook --delay 5

In [ ]:
%generate_report

In [ ]:
# %taegis investigations search-queries list

In [ ]:
# %taegis investigations search-queries stage

In [ ]:
# %taegis investigations create \
# --title "$INVESTIGATION_TITLE" \
# --key-findings "$TAEGIS_MAGIC_REPORT_FILENAME" \
# --priority MEDIUM \
# --type THREAT_HUNT \
# --status AWAITING_ACTION \
# --assignee-id "@customer" \
# --tenant $TENANT_ID \
# --region $REGION